In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [18]:
from utils import *

data = load_data()
len(data)

9906

In [19]:
dialog, dialog_id = pick_dialog(data, dialog_id='random', domain='all', exclusive=False)
print_dialog_goal(dialog, dialog_id)
print()
show_dialog_text(dialog)

[Dialog Id] PMUL1683.json

[User Goals]
You are planning your trip in Cambridge.
You are looking for a train. The train should go to stansted airport and should leave on wednesday.
The train should leave after 08:45 and should depart from cambridge.
Make sure you get arrival time.
You are also looking for a place to stay. The hotel should have a star of 2 and should include free wifi.
The hotel should include free parking and should be in the type of hotel.
Once you find the hotel you want to book it for 4 people and 3 nights starting from sunday.
Make sure you get the reference number.

[hotel]
{'book': {'day': 'sunday', 'invalid': False, 'people': '4', 'stay': '3'},
 'fail_book': {},
 'fail_info': {},
 'info': {'internet': 'yes', 'parking': 'yes', 'stars': '2', 'type': 'hotel'}}

[train]
{'fail_info': {},
 'info': {'day': 'wednesday',
          'departure': 'cambridge',
          'destination': 'stansted airport',
          'leaveAt': '08:45'},
 'reqt': ['arriveBy']}


User: I want a

## Engine

In [20]:
import engine

# TODO: find out why it does not work with GPT-4/GPT-4o even after removing the assertions
model_name = 'gpt-3.5-turbo'
run_result = engine.run(dialog, agent_type='react', agent_model=model_name, user_model=model_name)

================================================== Turn 1 ==================================================

User: I want a train going to stansted airport .


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Train Query
Action Input: destination: stansted airport

> Entering new SQLDatabaseChainWithCleanSQL chain...
destination: stansted airport
SQLQuery:SELECT "trainID", "departure", "destination", "leaveAt", "arriveBy", "price"
FROM train
WHERE destination = "stansted airport"
LIMIT 5;
SQLResult: [('TR4964', 'cambridge', 'stansted airport', '05:40', '06:08', '10.10 pounds'), ('TR7460', 'cambridge', 'stansted airport', '06:40', '07:08', '10.10 pounds'), ('TR4602', 'cambridge', 'stansted airport', '07:40', '08:08', '10.10 pounds'), ('TR0797', 'cambridge', 'stansted airport', '08:40', '09:08', '10.10 pounds'), ('TR5030', 'cambridge', 'stansted airport', '09:40', '10:08', '10.10 pounds')]
Answer:The trains going to Stansted Airport are TR4964, TR7

## Eval

In [22]:
from utils import DOMAINS
from evaluate import evaluate_by_domain

for domain in DOMAINS:
    if not dialog['goal'].get(domain):
        continue
    print('=' * 40)
    try:
        eval_result = evaluate_by_domain(domain, run_result, model=model_name)
    except Exception as e:
        msg = f'Run dialog failed as {e.__class__.__name__}: '
        print(colored(msg, 'red') + str(e))

[domain] hotel
[goal]
    info: {'internet': 'yes', 'type': 'hotel', 'stars': '2', 'parking': 'yes'}
    fail_info: {}
    book: {'people': '4', 'day': 'sunday', 'invalid': False, 'stay': '3'}
    fail_book: {}
[inform] complete: 0
    venue_name: none
    venue_info: "none" is not found in the "hotel" table.
[success] complete: None
    slot_values: None
[book] complete: 0
    refer_number: none
    book_record: "none" is not found in the "book_hotel" table.
[domain] train
[goal]
    info: {'leaveAt': '08:45', 'destination': 'stansted airport', 'day': 'wednesday', 'departure': 'cambridge'}
    reqt: ['arriveBy']
    fail_info: {}
[inform] complete: 1
[success] complete: 1
    slot_values: {'arriveBy': '10:08'}
[book] complete: None
    refer_number: None
    book_record: None
    train_info: None
